# MNIST Gabor-CNN
&emsp;&emsp;卷积神经网络（CNN）在图像识别领域取得了巨大成功，在CNN的框架设计中，用于从空域图像中提取图像特征的滤波器（filter）的优化完全依靠模型在深度神经网络中学习得到，并不包含hand-craft的人类先验信息。论文*Luan S , Zhang B , Chen C , et al. Gabor Convolutional Networks[J]. 2017.*，其尝试将模型自主学习filters和人类关于图像提取的先验知识相结合，以增强filters表现的稳健性。  
&emsp;&emsp;论文的做法是尝试将Gabor wavelets滤波器引入到模型当中，但并不直接将Gabor wavelets滤波器硬编码为底层滤波器，而是将Gabor wavelets滤波器与模型中可训练filters相结合得到增强后的filters用于与特征图（feature maps）进行后续的卷积操作。  
&emsp;&emsp;在本项目中，我将简单介绍Gabor滤波器在特征提取中的作用及基本原理，并利用tensoflow基本复现论文结果，展示tensorflow实现gabor-CNN应用于MNIST数据集中的代码具体细节，并将运行结果与论文结果进行对比讨论。本文将分以下点进行阐述：
- **1.卷积与特征提取**
- **2.gabor wavelets滤波器的特性**
- **3.gabor-CNN的基本框架及tensorflow实现**
- **4.MNIST上运行结果对比**

## 1.卷积与特征提取
&emsp;&emsp;无论是空域中的图像$f(x,y)$还是时域中的音频信号$f(t)$都是信号处理任务。在时空域进行特征提取，都可以对应到频域中的信号处理。在**空域**中，对图像进行`特征提取`实际上是寻找到图像对某个系统函数的`特征响应（response）`$g(x,y)$，这个系统函数就是我们所说的滤波器$h(x,y)$（filters），获取图像对于该系统函数的响应，即通过`卷积`运算$$g(x,y)=f(x,y)\otimes h(x,y)$$

&emsp;&emsp;为什么时空域中的卷积能够起到特征提取的效果？`卷积定理`函数卷积的傅里叶变换是函数傅里叶变换的卷积。将滤波器进行傅里叶变换后，与图像频谱相乘，就可以控制频谱特征的提取（方向，频率等）。而这一步可以通过在时域上的卷积得到。$$FT[f(x,y)\otimes h(x,y)]=F(u,v)\cdot H(u,v)=G(x,y)$$

&emsp;&emsp;由此可见滤波器的性质决定了图像对该滤波器的特征响应，那么根据我们希望提取的特征设计具有对应良好性质的滤波器就十分重要。Gabor滤波器就是十分适合于纹理特征提取的一类滤波器。

## 2.gabor wavelets滤波器的特征
&emsp;&emsp;gabor滤波器的特点及优势体现在：
- 提取特定方向（orientation）的正弦波
- 提取特定频率（frequency）的正弦波
- 提取局部特征（localized）
- 对光照不敏感

&emsp;&emsp;2D gabor wavelets滤波器/核表示为：$$\Psi_{u,v}=\frac{\Vert k_{u,v}\|^2}{\sigma^2}exp(-\frac{\| k_{u,v}\|^2 \|z\|^2}{2\sigma^2})[exp(ik_{u,v}z)-exp(-\sigma^2/2)]$$
- $u$：核方向（orientation）
- $v$：核尺度（scale），表示频率
- $z$：坐标$z=(x,y)$ 
- $\sigma$：高斯核半径，文章中取值为$2\pi$
- $k_{u,v}=k_ue^{ik_u}$：其中$k_u$控制滤波方向，$k_v$控制频率
- $k_v=k_{max}/f^{v-1}$：$k_{max}=\pi/2$为最大频率，$f=\sqrt{2}$为空间因子
- $k_u=u\pi/4$：$k_{u,v}=(k_v \cos k_u,k_v \sin k_u)$
- $exp(-\sigma^2/2)$：直流分量，模拟光照

&emsp;&emsp;gabor wavelets核由两部分组成，一个`二维正弦波`（控制方向和频率）以及`高斯窗口`（控制响应范围）叠加后具有了以下形状：
<img src="img/3d.jpg" height="70%" width="70%" title="Figure1: Gabor wavelets"></img>  
&emsp;&emsp;选取不同的参数对gabor核的影响：
<img src="img/gabor9.jpg" height="60%" width="60%"></img>
&emsp;&emsp;Gabor小波滤波器的频域响应。由于高斯核的傅里叶变换仍然是高斯核，复正弦（复指）的傅里叶变换是Dirac函数（冲激函数），所以Gabor的频响为两者卷积，Gabor滤波器在频域上也是局部的（对频谱加窗处理，而不是全局）。
<img src="img/fourier.jpg" height="60%" width="60%"></img>
&emsp;&emsp;Gabor filters应用于图片滤波，下图采用4个gabor filters提取一副棋盘。
<table>
  <tr>
    <td><img width="300px";src="img/checkerboard20.jpg"></td>
    <td><img width="300px";src="img/gabor10.jpg"></td>
  </tr>
</table>
以及采用水平的Gabor filter提取斑马图像信息。
<table>
  <tr>
    <td><img width="300px";src="img/zibrao.jpg"></td>
    <td><img width="300px";src="img/zibrag.jpg"></td>
  </tr>
</table>


## 3.gabor-CNN的基本框架及tensorflow实现
&emsp;&emsp;本文的gabor-CNN网络包含4个卷积层和2个全连接层。其结构图如下：
<img src="img/gcnn.jpg" width="80%" height="80%"></img>
由于需要确保每层卷积后的feature maps都包含由$U=4$个gabor filters增强后的filters分别卷积的结果，因此需要对原本的2D图像进行**升维**，将特征提取的方向性作为一个新的维度；那么将采用3D卷积-->`tf.nn.conv3d`。相关记号表示：暂时先忽略batch_size这一维度。
- feature maps: [$N, U, H, W$]=[$channels, orientations, height, width$]
- trainable filters/kernel: [$N_{j+1}, N_j, H, W$]=[$output \; channels, input \; channels, height, width$]
- gabor filter banks: 全局常量$gabor(u,v)$-->[$U, H, W$]每一卷积层中，$v$是固定的，$u=0,1,2,3$
- GoFs：经过gabor增强过后的filters [$N_{j+1},U,N_j,H,W$]

### 3.1gabor filter banks:
$$GoFs_{i,u}=C_{i,u}=\left[ \begin{matrix} K_{i,1}\circ G(u,v)\\ K_{i,2}\circ G(u,v) \\ ... \\ K_{i,N_j}\circ G(u,v) \end{matrix} \right]$$
    $$i=1,2,...,N_{j+1};\; u=0,1,2,3;\; v在一个卷积层内为常数$$
    $$\circ为Hadamard积;\;K为trainable \;filters; \; G(u,v)为2D\;gabor \; filters$$
<img src="img/gof.jpg" width="50%" height="50%"></img>

在应用于实际处理中，一般只用Gabor wavelets的实部$$\Psi_{u,v}=\frac{\Vert k_{u,v}\|^2}{\sigma^2}exp(-\frac{\| k_{u,v}\|^2 \|z\|^2}{2\sigma^2})[\cos(k_v\cos k_ux+k_v\sin k_uy)-exp(-\sigma^2/2)]$$
回顾Gabor wavelets的各部分参数：
- $u$：orientation
- $v$：scale
- $z$：坐标$z=(x,y)$ 
- $\sigma$：$2\pi$
- $k_{u,v}=k_ue^{ik_u}$：其中$k_u$控制滤波方向，$k_v$控制频率
- $k_v=k_{max}/f^{v-1}$：$k_{max}=\pi/2$，$f=\sqrt{2}$
- $k_u=u\pi/4$：$k_{u,v}=(k_v \cos k_u,k_v \sin k_u)$
- $exp(-\sigma^2/2)$：直流分量

In [ ]:
    def get_gabor_banks(self, u, v, w, h, sigma=math.pi*2):
        """
        得到一层中的gabor filters
        """
        kmax = math.pi / 2
        f = math.sqrt(2)
        sqsigma = sigma ** 2
        postmean = math.exp(-sqsigma / 2)
        if h != 1:
            gfilter = np.zeros([u, h, w], dtype=np.float32)
            for i in range(u):
                theta = i / u * math.pi  # k_u
                k = kmax / f ** (v - 1)  # k_v
                xymax = -1e30
                xymin = 1e30
                for y in range(h):
                    for x in range(w):
                        y_ = y + 1 - ((h + 1) / 2)
                        x_ = x + 1 - ((w + 1) / 2)
                        tmp1 = math.exp(- (k ** 2 * (x_ ** 2 + y_ ** 2) / (2 * sqsigma)))  # Gaussian kerbel
                        tmp2 = math.cos(k * math.cos(theta) * x_ + k * math.sin(theta) * y_) - postmean  # complex sinusoid
                        gfilter[i][y][x] = k ** 2 *tmp1 *tmp2 / sqsigma
                        xymax = max(xymax, gfilter[i][y][x])
                        xymin = max(xymin, gfilter[i][y][x])
                gfilter[i] = (gfilter[i] - xymin) / (xymax - xymin)
        else:
            gfilter = np.ones([u, h, w], dtype=np.float32)
        return gfilter


### 3.2 GoFs与feature maps的卷积
&emsp;&emsp;回顾第$j+1$层输入feature maps $F^j$的维度[$N_j,U,H,W$]，输出feature maps $F^{j+1}$维度为[$N_{j+1},U,H,W$]。
- 输出的第一维度（$N^{j+1}，channels$）中每个元素$i$是由第i个$GoF^j$与输入卷积得到
- 输出的第二维度（$U，orientations$）中每个元素$u$是由gabor filter第u个方向增强后得到的$C^j_{i,u}$与输入卷积得到。
$$F^{j+1}_{i,u}=\sum_{u'=1}^{U}\sum^{N_j}_{n=1}F^j_{n}\otimes C^j_{i,u,n}$$
  - $F^j_{n}$是3D tensor [$U,H,W$]，虽然$C^j_{i,u,n}$是2D tensor，但可以认为具有[$1,H,W$]的3D结构；两者卷积运算后得到的tensor为3D[$U,H,W$]。由于该一次卷积只能生成输出第二维度的一个元素，因此需要在维度U上加和降维，$u'=1,2,3,4$代表卷积后tensor的第一维度。
  - $\sum_{u'=1}^{U}$求和降维后得到2D tensor [$H,W$]。对于输入的每个通道$n=1,2,...,N_{j}$都得到这样的2D tensor并将其求和得到$F^{j+1}_{i,u}$
  - 为了得到输出的一个通道feature map$F^{j+1}_{i}$，需要用$U=4$次上述卷积过程，然后将其结果堆叠；每次卷积的卷积核来自gabor filters不同方向，含义为在输入中分别提取$U=4$个方向的特征作为输出的第二维度。

In [ ]:
    def add_conv_layer(self, input, filter_shape,v,data_format="NDHWC",stride_height=1,stride_width=1,stride_depth=1,padding="VALID" ):
        self.num_conv += 1
        with self.graph.as_default():
            with tf.name_scope("conv{}".format(self.num_conv)):
                with tf.name_scope("conv_kernel"):
                    w_shape = filter_shape.copy()
                    w_shape.pop(0)
                    w_shape.reverse()
                    # shape: [out_channels, in_channels, height, width]
                    _W = tf.Variable(tf.truncated_normal(w_shape, stddev=0.1))
                    # 得到对应gabor filters, ahspe: [U, height, width]
                    gabor = self.gabor_filter_banks.get((self.U, v), None)
                    if not gabor:
                        gabor = self.get_gabor_banks(u=self.U,
                                                     v=v, h=filter_shape[1],
                                                     w=filter_shape[2])
                        self.gabor_filter_banks.update({(self.U, v): gabor})
                    gabor = tf.constant(gabor)
                    gabor_W = []
                    for o in range(w_shape[0]):
                        inner = []
                        # Hadamard积求得一个GoF
                        for g in range(self.U):
                            inner.append(_W[o] * gabor[g])
                        inner = tf.stack(inner, axis=0)
                        gabor_W.append(inner)
                    # shape: [out_channels, U, in_channels, height, width]
                    # 堆叠结果得到5D的GoFs
                    gabor_W = tf.stack(gabor_W, axis=0, name="filter")
                    # shape: [out_channels, U, in_channels, depth, height, width]
                    # 需要对GoFs进行升维depth=1,卷积核为[1, height, width]
                    gabor_W_ = tf.expand_dims(gabor_W, axis=3)
                    # 维度交换[U, depth, height, width, in_channels, out_channels]
                    gabor_W_ = tf.transpose(gabor_W_, perm=[1, 3, 4, 5, 2, 0])

                    # 需要多次卷积并将卷积结果进行堆叠组合
                    maps = []
                    for i in range(self.U):
                        conv_ = tf.nn.conv3d(input,
                                            filter=gabor_W_[i],
                                            strides=[1, stride_depth, stride_height, stride_width, 1],
                                            padding=padding,
                                            data_format=data_format,
                                            )
                        # 在depth维度上求和，但保留该维度
                        # [batch_size, 1, height, width, out_channels]
                        conv_ = tf.reduce_sum(conv_, axis=1, keepdims=True)
                        maps.append(conv_)
                    # [batch_size, U, height, width, out_channels]
                    conv3 = tf.concat(maps, axis=1)
        return conv3


### 3.3 Batch normalization以及Pooling
&emsp;&emsp;这部分操作与CNN基本相同，无结构创新。基本结构为`conv outpt`-->`Batch normalization`-->`max pooling`-->`ReLu`。只需要注意池化操作只在height, width上进行，depth上（锁死为U=4）不进行池化操作。

In [ ]:
    def add_batch_normalization_layer(self, input):
        with self.graph.as_default():
            with tf.name_scope("batch_normalized"):
                # 保留depth维度上的独立性
                batch_mean, batch_var = tf.nn.moments(input,
                                                      [0, 2, 3],
                                                      keep_dims=True)
                offset = tf.Variable(tf.zeros(batch_mean.shape.as_list()))
                scale = tf.Variable(tf.ones(batch_mean.shape.as_list()))
                bn = tf.nn.batch_normalization(input,
                                               mean=batch_mean,
                                               variance=batch_var,
                                               offset=offset,
                                               scale=scale,
                                               variance_epsilon=1e-4)
        return bn
    
        def add_pool_layer(self, hidden, pool_type, ksize, strides, activation=tf.nn.relu, padding="VALID"):
        with self.graph.as_default():
            with tf.name_scope(pool_type):
                if pool_type == "max_pool":
                    _pool = tf.nn.max_pool3d(hidden,
                                             ksize=ksize,
                                             strides=[1, *strides, 1],
                                             padding=padding)
                elif pool_type == "avg_pool":
                    _pool = tf.nn.avg_pool3d(hidden,
                                             ksize=ksize,
                                             strides=[1, *strides, 1],
                                             padding=padding)
                else:
                    _pool = hidden
                _pool = activation(_pool)
                return _pool


### 3.4 Fully connected layers及output layer
&emsp;&emsp;此部分内容与CNN结构也基本相同，但是在最后一层卷积输出到全连接层输入之间还进行了一个depth维度上的max pooling。  
&emsp;&emsp;最后一层卷积的输出为[$?, 4, 1, 1, 160$]，并不直接拉长，而是在depth上进行max_pooling，结果为[$?,1,1,1,160$]。降维后得到[$?,160$]的全连接层输入

## 4.训练参数及结果
&emsp;&emsp;本次训练数据集来源采用tensorflow-datasets导入
```python
import tensorflow_dataset as tfds
mnist_train = tfds.load(name="mnist", data_dir=TRAIN_DIR, split=tfds.Split.TRAIN, download=True)
mnist_test = tfds.load(name="mnist", data_dir=TRAIN_DIR, split=tfds.Split.TEST, download=True)
```
&emsp;&emsp;由于该包暂时不提供分隔验证集，因此训练集大小为60,000，测试集大小10,000。（论文中训练集50,000，切分10,000作为验证集）。模型结构、参数量与论文所述相同。但由于时间所限迭代的次数远少于论文。

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-36ox{color:#fe0000;border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-kiyi{font-weight:bold;border-color:inherit;text-align:left}
.tg .tg-xldj{border-color:inherit;text-align:left}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-xldj"></th>
    <th class="tg-kiyi">原论文</th>
    <th class="tg-kiyi">复现结果</th>
  </tr>
  <tr>
    <td class="tg-xldj">模型</td>
    <td class="tg-xldj">GCNN</td>
    <td class="tg-xldj">GCNN</td>
  </tr>
    <tr>
    <td class="tg-xldj">input size</td>
    <td class="tg-xldj">1*32*32</td>
    <td class="tg-xldj">1*28*28</td>
  </tr>
  <tr>
    <td class="tg-xldj">batch size</td>
    <td class="tg-xldj">128</td>
    <td class="tg-xldj">128</td>
  </tr>
  <tr>
    <td class="tg-0pky">activation</td>
    <td class="tg-0pky">ReLu</td>
    <td class="tg-0pky">ReLu</td>
  </tr>
  <tr>
    <td class="tg-xldj">训练集</td>
    <td class="tg-xldj">50,000</td>
    <td class="tg-xldj">60,000</td>
  </tr>
  <tr>
    <td class="tg-xldj">测试集</td>
    <td class="tg-xldj">10,000</td>
    <td class="tg-xldj">10,000</td>
  </tr>
  <tr>
    <td class="tg-xldj">优化器</td>
    <td class="tg-xldj">AdamOptimizer</td>
    <td class="tg-xldj">AdamOptimizer</td>
  </tr>
  <tr>
    <td class="tg-xldj">laerning rate</td>
    <td class="tg-xldj">初始0.001，exponential decay, 半衰期为25个epochs</td>
    <td class="tg-xldj">初始0.001，exponential decay, 每200个batch衰减为0.9</td>
  </tr>
  <tr>
    <td class="tg-xldj">iterations</td>
    <td class="tg-xldj">200个epochs</td>
    <td class="tg-xldj">2000个batches，约4.25个epochs</td>
  </tr>
  <tr>
    <td class="tg-0pky">test error rate</td>
    <td class="tg-36ox"><font color="red">0.56%</font></td>
    <td class="tg-36ox"><font color="red">0.70%</font></td>
  </tr>
</table>
&emsp;&emsp;在tensorboard中记录准确率和代价函数随迭代的变化。
<table>
  <tr>
    <td><img width="370px";src="img/acc.jpg"></td>
    <td><img width="370px";src="img/loss.jpg"></td>
  </tr>
</table>